In [181]:
import pandas as pd
import json
import numpy as np

In [182]:
with open ('../backend/dev/resources/files/organisations.json', 'r') as orgs:
    old = json.loads(orgs.read())

In [183]:
old = pd.DataFrame(old)
old['abbr'] = old['name'].apply(lambda x: '' if len(x.split('(')) == 1 else x.split('(')[1].replace(')',''))
old['lname'] = old['name'].apply(lambda x: x.split('(')[0].lower())

In [184]:
new = pd.read_csv('./new_organisation.csv')
new['abbr'] = new['name'].apply(lambda x: '' if len(x.split('(')) == 1 else x.split('(')[1].replace(')',''))
new['lname'] = new['name'].apply(lambda x: x.split('(')[0].lower())

In [185]:
merged = pd.merge(new, old, on="lname", how="outer", validate="one_to_one").replace([np.nan], [None])
merged['name'] = merged.apply(lambda x: x['name_x'] if x['name_x'] else x['name_y'], axis=1)
merged = merged[['name','country','country_group','type','url']]
merged['urls'] = merged['url'].apply(lambda x: [b.strip() for b in x.split('\n') if b.strip() != ''] if x else None)
merged = merged[['name','country','country_group','type','urls']].sort_values('name')

In [186]:
merged.to_csv('organisation_merged.csv', index=False)

In [187]:
unlisted = merged[merged['type'].isna()]

In [188]:
unlisted.to_csv('organisation_unlisted.csv')